In [35]:
from random import seed
from random import random
from math import exp
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [77]:
# ANN
def initialize_network(n_inputs, n_hidden, n_outputs):
  network = list()
  hidden_layer = [{'weights':[random() for i in range(n_inputs+1)]} for i in range(n_hidden)]        # random weights of hidden layer
  network.append(hidden_layer)
  output_layer = [{'weights':[random() for i in range(n_hidden+1)]} for i in range(n_outputs)]       # random weights of output layer
  network.append(output_layer)
  return network

# Activation Function
def activate(weights, inputs):
	activation = weights[-1]
	for i in range(len(weights)-1):
		activation += weights[i] * inputs[i]
	return activation

# Sigmoid Function
def transfer(activation):
 return 1.0 / (1.0 + exp(-activation))

# g'(x) = 1 * (1-x)
def transfer_derivative(output):
 return output * (1.0 - output)

# Forward propagation
def forward_propagate(network, row):    # row = list of inputs
	inputs = row
	for layer in network:
		new_inputs = []
		for neuron in layer:
			activation = activate(neuron['weights'], inputs)
			neuron['output'] = transfer(activation)
			new_inputs.append(neuron['output'])
		inputs = new_inputs
	return inputs                       # array of outputs

# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(neuron['output'] - expected[j])
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

# Update network weights with error
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-3]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] -= l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] -= l_rate * neuron['delta']

# Training the network
def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            inputs = row[:-n_outputs]  # Extract input features
            expected = row[-n_outputs:]  # Extract one-hot encoded expected output
            outputs = forward_propagate(network, inputs)
            sum_error += sum([(expected[i] - outputs[i])**2 for i in range(len(expected))])
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)  # Update weights using the entire row
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

In [61]:
df = pd.read_csv('Iris.csv')
df.drop(columns=['Id'],inplace=True)
one_hot_encoded = pd.get_dummies(df['Species'], prefix='category')
df_encoded = pd.concat([df, one_hot_encoded], axis=1)
df_encoded.drop(columns=['Species'],inplace = True)

inputs = df_encoded.iloc[:, :-3].values   # Input features
outputs = df_encoded.iloc[:, 4:].values   # Output column


In [156]:
seed(1)
#train = 70%   test = 30%
train_data, test_data = train_test_split(df_encoded, test_size=0.3, random_state=42)

dataset = train_data.iloc[:,:].values
n_inputs = len(dataset[0]) - 3
n_outputs = 3
network = initialize_network(n_inputs, 6 , n_outputs)
train_network(network, dataset, 0.1, 30, n_outputs)   # ANN, dataset ,Learning rate , iterations , nuber of output columns
for layer in network:
 print(layer)

>epoch=0, lrate=0.100, error=157.598
>epoch=1, lrate=0.100, error=112.496
>epoch=2, lrate=0.100, error=82.037
>epoch=3, lrate=0.100, error=71.269
>epoch=4, lrate=0.100, error=70.688
>epoch=5, lrate=0.100, error=68.704
>epoch=6, lrate=0.100, error=65.033
>epoch=7, lrate=0.100, error=60.961
>epoch=8, lrate=0.100, error=56.653
>epoch=9, lrate=0.100, error=52.776
>epoch=10, lrate=0.100, error=49.655
>epoch=11, lrate=0.100, error=47.253
>epoch=12, lrate=0.100, error=45.404
>epoch=13, lrate=0.100, error=43.940
>epoch=14, lrate=0.100, error=42.730
>epoch=15, lrate=0.100, error=41.680
>epoch=16, lrate=0.100, error=40.723
>epoch=17, lrate=0.100, error=39.813
>epoch=18, lrate=0.100, error=38.919
>epoch=19, lrate=0.100, error=38.019
>epoch=20, lrate=0.100, error=37.098
>epoch=21, lrate=0.100, error=36.146
>epoch=22, lrate=0.100, error=35.157
>epoch=23, lrate=0.100, error=34.130
>epoch=24, lrate=0.100, error=33.066
>epoch=25, lrate=0.100, error=31.970
>epoch=26, lrate=0.100, error=30.850
>epoch=27

In [157]:
def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))  # Return the index of the neuron with the highest output

# Evaluate the accuracy of the model using the test data
def evaluate_model(network, test_data):
    correct_predictions = 0
    total_samples = len(test_data)
    for row in test_data:
        inputs = row[:-3]  # Extract input features
        expected = np.argmax(row[-3:])  # Extract the index of the expected output neuron
        prediction = predict(network, inputs)
        if prediction == expected:
            correct_predictions += 1
    accuracy = (correct_predictions / total_samples) * 100
    return accuracy

# Use the test data to evaluate the model accuracy
test_dataset = test_data.values
accuracy = evaluate_model(network, test_dataset)
print("Accuracy:", accuracy)

Accuracy: 97.77777777777777
